# Bayesian SIR Model with Change Points

In [1]:
import numpy as np
from scipy.stats import binom, gamma, beta, expon, poisson, uniform, bernoulli
from joblib import Parallel, delayed
from scipy.special import gamma as gammaFunc
import random

import pandas as pd

import matplotlib.pyplot as plt

import tqdm

import itertools

## Data Simulation

In [2]:
def sim_dataset(chg_pt, scenarios, T, S0, I0, R0, n_datasets):

    N = S0 + I0 + R0
    n_sc = scenarios.shape[0]

    # create array of transmission and removal rate parameters at each time step
    beta  = np.array([[scenarios[i,(chg_pt <= t+1).sum(),0] for t in range(T)] for i in range(n_sc)])
    gamma = np.array([[scenarios[i,(chg_pt <= t+1).sum(),1] for t in range(T)] for i in range(n_sc)])


    Delta_I = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)
    Delta_R = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)
    S       = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)
    I       = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)
    R       = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)

    Delta_I[:,:,0] = binom.rvs(S0, 1-np.exp(-beta[:,0]*I0/N), size=(n_datasets, n_sc))
    Delta_R[:,:,0] = binom.rvs(I0, gamma[:,0], size=(n_datasets, n_sc))
    S[:,:,0]       = S0 - Delta_I[:,:,0]
    I[:,:,0]       = I0 + Delta_I[:,:,0] - Delta_R[:,:,0]
    R[:,:,0]       = R0 + Delta_R[:,:,0]

    for t in range(1, T):
        Delta_I[:,:,t] = binom.rvs(S[:,:,t-1], 1-np.exp(-beta[:,t]*I[:,:,t-1]/N))
        Delta_R[:,:,t] = binom.rvs(I[:,:,t-1], gamma[:,t])
        S[:,:,t]       = S[:,:,t-1] - Delta_I[:,:,t]
        I[:,:,t]       = I[:,:,t-1] + Delta_I[:,:,t] - Delta_R[:,:,t]
        R[:,:,t]       = R[:,:,t-1] + Delta_R[:,:,t]

    return Delta_I, Delta_R, S, I, R

In [3]:
def plot_SRI(S, I, R, sc=0, d=None, start_cond=(999_950, 50, 0), tot=1_000_000, time=100):

    if d is None:
        S = np.expand_dims(np.mean(S, axis=0), 0)
        I = np.expand_dims(np.mean(I, axis=0), 0)
        R = np.expand_dims(np.mean(R, axis=0), 0)
        d = 0

    S = np.concatenate([[start_cond[0]], S[d,sc]])
    I = np.concatenate([[start_cond[1]], I[d,sc]])
    R = np.concatenate([[start_cond[2]], R[d,sc]])

    # plot
    fig, ax = plt.subplots()

    y = np.vstack([S, I, R])
    ax.stackplot(np.arange(time+1), y/tot, labels=["S","I","R"], alpha=0.8)

    ax.set_xlabel("Day")
    ax.set_ylabel("Proportion")
    ax.set_xticks(np.concatenate([[0], np.arange(25, time+1, 25)]))
    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.legend(loc="upper right")

    plt.show()

### Example

In [4]:
T = 100
N = 1_000_000

chg_pt = np.array([26, 51, 76])

sc_1 = [(0.3, 0.05), (0.4, 0.15), (0.25, 0.2),  (0.2,  0.25)]
sc_2 = [(0.4, 0.1),  (0.4, 0.25), (0.25, 0.25), (0.25, 0.4) ]
sc_3 = [(0.5, 0.1),  (0.3, 0.3),  (0.4,  0.2),  (0.2,  0.4) ]
scenarios = np.array([sc_1, sc_2, sc_3])

S0 = N-50
I0 = 50
R0 = 0

n_datasets = 100

In [5]:
Delta_I, Delta_R, S, I, R =  sim_dataset(chg_pt, scenarios, T, S0, I0, R0, n_datasets)
#for i in range(len(scenarios)):
 #   print(f"Scenario {i+1}:")
#    plot_SRI(S, I, R, sc=i)

## Gibbs Sampling

In [8]:
S_obs = np.concatenate(([S0],S[0,0]))
I_obs = np.concatenate(([I0],I[0,0]))
R_obs = np.concatenate(([R0],R[0,0]))
PI_obs = I_obs/N
N_infect_obs = np.concatenate(([I0],Delta_I[0,0]))
N_recovery_obs = np.concatenate(([R0],Delta_R[0,0]))

data = pd.DataFrame({
        'susceptible': S_obs,
        'infects': I_obs,
        'recovered': R_obs,
        'PI': PI_obs,
        'deltaI': N_infect_obs,
        'deltaR': N_recovery_obs
    })

In [9]:
data

,susceptible,infects,recovered,PI,deltaI,deltaR
0,999950,50,0,0.000050,50,0
1,999929,70,1,0.000070,21,1
2,999907,91,2,0.000091,22,1
3,999877,116,7,0.000116,30,5
4,999845,143,12,0.000143,32,5
...,...,...,...,...,...,...
96,158253,16,841731,0.000016,0,12
97,158252,17,841731,0.000017,1,0
98,158252,16,841732,0.000016,0,1
99,158252,12,841736,0.000012,0,4


In [14]:
def calculate_contact_hat_parallel(pp_lambda_t):
    pp, lambda_t = pp_lambda_t
    return np.sum(poisson.ppf(pp, lambda_t))

def gibbs_sampling(data, samples=10000, T_max=100, burnin=1000, thinning=10):
    
    ############# data
    # Data is expected to be a dataframe with 101 rows (100 steps + initial one)
    # OBS: the initial row should display the starting values 
    #      for Susceptible (S), Infected (I), and Recovered (R), with zero values in the 
    #      columns representing changes (deltas) over time.
    
    I_obs = data["infects"].values
    S_obs = data["susceptible"].values
    PI_obs = data["PI"].values
    N_infect_obs = data["deltaI"].values[1:]
    N_recovery_obs = data["deltaR"].values[1:]
    

    #######################
    ##  HYPERPARAMETERS  ##
    #######################
    
    p_a = 1/T_max**4
    p_b = 2 - p_a
    
    b_shape = 0.1
    b_rate = 0.1
    r_shape = 0.1
    r_rate = 0.1
    
    gamma_b_shape = gammaFunc(b_shape)
    gamma_r_shape = gammaFunc(r_shape)
    

    ######################
    ##  INITIALIZATION  ##
    ######################
    
    ##### delta
    Delta_hat = np.zeros(shape=T_max, dtype=int)
    Delta_hat[0] = 1
    Stage_hat = np.cumsum(Delta_hat, dtype=int)-1
    K_hat     = np.sum(Delta_hat, dtype=int)

    ##### b and r
    b_hat    = gamma.rvs(a=b_shape, scale=1/b_rate, size=K_hat)
    r_hat    = gamma.rvs(a=r_shape, scale=1/r_rate, size=K_hat)
    
    ##### beta and gamma
    beta_hat    = expon.rvs(scale=1/b_hat[Stage_hat[0]], size=T_max)
    gamma_hat   = beta.rvs(a=r_hat[Stage_hat[0]], b=1, size=T_max)
    
    
    ##### Infetious contacts
    # Given lambda_t, this code computes the probability that a given infect will contact a susceptible individual.
    # Then the number of conctacts between these two classes is computed in parallel.
    # As a result, contact_hat is the number of contacts between new infects and susceptible individuals.
    lambda_t    = beta_hat*PI_obs[:-1]
    p_upper     = 1 - poisson.cdf(0, lambda_t) 
    pp          = [uniform.rvs(size=N_infect_obs[t]) * p_upper[t] + (1 - p_upper[t]) for t in range(T_max)]
    contact_hat = np.array(Parallel(n_jobs=-1)(delayed(calculate_contact_hat_parallel)((pp[t], lambda_t[t],)) for t in range(T_max)))
    
    
    beta_hat    = gamma.rvs(a=contact_hat + 1, scale=1/(b_hat[Stage_hat] + PI_obs[:-1]*S_obs[:-1]))
    gamma_hat   = beta.rvs(a=N_recovery_obs + r_hat[Stage_hat], b=1 + I_obs[:-1] - N_recovery_obs)
    lambda_t    = beta_hat * PI_obs[:-1]
    p_upper     = 1 - poisson.cdf(0, lambda_t)
    pp          = [uniform.rvs(size=N_infect_obs[t]) * p_upper[t] + (1 - p_upper[t]) for t in range(T_max)]
    contact_hat = np.array(Parallel(n_jobs=-1)(delayed(calculate_contact_hat_parallel)((pp[t], lambda_t[t],)) for t in range(T_max)))
    
    
    print("Initialization:\n")
    print("Delta_hat:",Delta_hat)
    print("b_hat:",b_hat)
    print("r_hat:",r_hat)
    print("beta_hat:",beta_hat)
    print("gamma_hat:",gamma_hat)
    print("contact_hat:",contact_hat)
    print("----------------------------------------------------")
    print("----------------------------------------------------")
    

    ################
    ##  SAMPLING  ##
    ################
    
    Delta_all = []
    Stage_all = []
    b_all     = []
    r_all     = []
    beta_all  = []
    gamma_all = []
    
    for step in tqdm.tqdm(range(samples)):
        
        ###### delta_hat sampling
        ## add–delete–swap - proposal step
        # -1 delete
        # 0 swap
        # +1 add
                
        if K_hat==1:
            change_type = 1
        
        elif K_hat==T_max:
            change_type = -1

        else:
            change_type = np.random.choice([-1, 0, 1])
            
        Delta_hat_candidate = Delta_hat.copy()
        
        if change_type != 0:
            
            if change_type == 1:
                possible_change_indices = np.where(Delta_hat[1:] == 0)[0]+1
            elif change_type == -1:
                possible_change_indices = np.where(Delta_hat[1:] == 1)[0]+1
            index_to_change = np.random.choice(possible_change_indices)

            Delta_hat_candidate[index_to_change] = 1 - Delta_hat_candidate[index_to_change]
            Stage_hat_candidate = np.cumsum(Delta_hat_candidate)-1

            if change_type == 1:
                possible_change_indices_candidate = np.where(Delta_hat_candidate[1:] == 1)[0] + 1
                phase_original  = np.array([Stage_hat[index_to_change]])
                phase_candidate = np.array([Stage_hat[index_to_change], Stage_hat_candidate[index_to_change]])
            elif change_type == -1:
                possible_change_indices_candidate = np.where(Delta_hat_candidate[1:] == 0)[0] + 1
                phase_original  = np.array([Stage_hat[index_to_change], Stage_hat_candidate[index_to_change]])
                phase_candidate = np.array([Stage_hat_candidate[index_to_change]])
        
            logp_original = 0
            logp_candidate = 0

            for i in phase_original:
                L_i_original = np.where(Stage_hat == i)[0]

                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + b_shape)) \
                               - np.log(gamma_b_shape) + b_shape * np.log(b_rate) \
                               - (b_shape + len(L_i_original)) * np.log(b_rate + np.sum(beta_hat[L_i_original]))
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + r_shape)) \
                               - np.log(gamma_r_shape) + r_shape * np.log(r_rate) \
                               - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
                logp_original -= np.sum(np.log(np.arange(1, len(L_i_original))))

            for i in phase_candidate:
                L_i_candidate = np.where(Stage_hat_candidate == i)[0]

                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + b_shape)) \
                                - np.log(gamma_b_shape) + b_shape * np.log(b_rate) \
                                - (b_shape + len(L_i_candidate)) * np.log(b_rate + np.sum(beta_hat[L_i_candidate]))
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + r_shape)) \
                                - np.log(gamma_r_shape) + r_shape * np.log(r_rate) \
                                - (r_shape + len(L_i_candidate)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_candidate])))
                logp_candidate -= np.sum(np.log(np.arange(1, len(L_i_candidate))))

            logp_candidate += np.log(p_a / p_b) + np.log((3 - 2 * (K_hat == 1)) \
                            * len(possible_change_indices) / (3 - 2 * ((K_hat + 1) == T_max)) / len(possible_change_indices_candidate))

            ratio = np.exp(min([0, logp_candidate - logp_original]))

        elif change_type == 0:
            
            possible_change_indices = np.where(np.abs(Delta_hat[1:-1] - Delta_hat[2:]) == 1)[0]+1
            index_to_change = np.random.choice(possible_change_indices, 1)

            Delta_hat_candidate[index_to_change + np.array([0, 1])] = Delta_hat_candidate[index_to_change + np.array([1, 0])]
            Stage_hat_candidate = np.cumsum(Delta_hat_candidate)-1
   
            possible_change_indices_candidate = np.where(np.abs(Delta_hat_candidate[1:-1] - Delta_hat_candidate[2:]) == 1)[0]+1
        
            phase = np.array([Stage_hat[index_to_change], Stage_hat_candidate[index_to_change]])
    
            logp_original = 0
            logp_candidate = 0
        
            for i in phase:
                L_i_original = np.where(Stage_hat == i)[0]
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + b_shape)) \
                               - np.log(gamma_b_shape) + b_shape * np.log(b_rate) \
                               - (b_shape + len(L_i_original)) * np.log(b_rate + np.sum(beta_hat[L_i_original]))
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + r_shape)) \
                               - np.log(gamma_b_shape) + r_shape * np.log(r_rate) \
                               - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
                logp_original -= np.sum(np.log(np.arange(1, len(L_i_original))))

                L_i_candidate = np.where(Stage_hat_candidate == i)[0]
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + b_shape)) \
                                - np.log(gamma_b_shape) + b_shape * np.log(b_rate) \
                                - (b_shape + len(L_i_candidate)) * np.log(b_rate + np.sum(beta_hat[L_i_candidate]))
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + r_shape)) \
                                - np.log(gamma_b_shape) + r_shape * np.log(r_rate) \
                                - (r_shape + len(L_i_candidate)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_candidate])))
                logp_candidate -= np.sum(np.log(np.arange(1, len(L_i_candidate))))

            ratio = np.exp(min([0, logp_candidate - logp_original + np.log(len(possible_change_indices) / len(possible_change_indices_candidate))]))
        
        cxx = np.random.binomial(1, ratio)      
        if cxx == 1:
            Delta_hat = Delta_hat_candidate
            Stage_hat = Stage_hat_candidate
        
        
        ##### b and r sampling
            
        K_hat = np.sum(Delta_hat)
        
        b_hat = np.zeros(K_hat)
        r_hat = np.zeros(K_hat)
        for i in range(K_hat):
            L_i = np.where(Stage_hat == i)[0]
            b_hat[i] = gamma.rvs(size=1, a=(b_shape + len(L_i)), scale=1/(b_rate + np.sum(beta_hat[L_i])))
            r_hat[i] = gamma.rvs(size=1, a=(r_shape + len(L_i)), scale=1/(r_rate + np.sum(-np.log(gamma_hat[L_i]))))
    
        
        ##### beta and gamma sampling

        beta_hat    = gamma.rvs(a=contact_hat + 1, scale=1/(b_hat[Stage_hat] + PI_obs[:-1]*S_obs[:-1]))
        gamma_hat   = beta.rvs(a=N_recovery_obs + r_hat[Stage_hat], b=1 + I_obs[:-1] - N_recovery_obs)
        lambda_t    = beta_hat * PI_obs[:-1]
        p_upper     = 1 - poisson.cdf(0, lambda_t)
        pp          = [uniform.rvs(size=N_infect_obs[t]) * p_upper[t] + (1 - p_upper[t]) for t in range(T_max)]
        contact_hat = np.array(Parallel(n_jobs=-1)(delayed(calculate_contact_hat_parallel)((pp[t], lambda_t[t],)) for t in range(T_max)))
    
       


        if step % 100 == 0 and step != 0:
            print("\nStep:",step)
            print("Delta_hat:",Delta_hat)
            print("b_hat:",b_hat)
            print("r_hat:",r_hat)
            print("beta_hat:",beta_hat)
            print("gamma_hat:",gamma_hat)
            print("----------------------------------------------------")
            
        Delta_all.append(Delta_hat)
        Stage_all.append(Stage_hat)
        b_all.append(b_hat)
        r_all.append(r_hat)
        beta_all.append(beta_hat)
        gamma_all.append(gamma_hat)
  
    # Creazione del DataFrame
    MCMC_chain = {
        'Delta': Delta_all[burnin::thinning],
        'Stage': Stage_all[burnin::thinning],
        'b': b_all[burnin::thinning],
        'r': r_all[burnin::thinning],
        'beta': beta_all[burnin::thinning],
        'gamma': gamma_all[burnin::thinning]
            }
    
    return MCMC_chain

In [15]:
MCMC_chain=gibbs_sampling(data)

Initialization:

Delta_hat: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
b_hat: [3.66435366]
r_hat: [5.28772115]
beta_hat: [0.39300264 0.2839295  0.28402081 0.27767568 0.30905174 0.23441698
 0.27620669 0.26814828 0.28350367 0.33375794 0.33601871 0.27308276
 0.32821474 0.29377536 0.31365687 0.28849249 0.29925023 0.29635672
 0.31966484 0.28421601 0.30926008 0.30358806 0.29202708 0.28222269
 0.31038797 0.40111825 0.40068897 0.4007103  0.39631655 0.39774371
 0.40252078 0.40137069 0.39606779 0.39947228 0.39769244 0.40124507
 0.39482228 0.40931351 0.39020463 0.38664362 0.39452047 0.38484885
 0.38848808 0.37973352 0.39019378 0.38266595 0.38357959 0.38983337
 0.39707063 0.38863063 0.24472244 0.24980577 0.24956037 0.26477319
 0.25889606 0.24948481 0.26173641 0.24867855 0.24706602 0.2628349
 0.2441037  0.26381449 0.25080387 0.24973763 0.25

  0%|                                            | 5/10000 [00:05<2:54:03,  1.04s/it]


KeyboardInterrupt: 